In [ ]:
%pip install tiktoken

In [ ]:
%load_ext dotenv
%dotenv /notebooks/chainlang/my.env

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


INIT MODEL

In [ ]:

llm = ChatOpenAI(model_name="gpt-4") 
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

LOAD DOCUMENTS

In [ ]:
repo_path = "/notebooks/fastai-chap4/"
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".php",".js",".tsx",".ipynb",".py"],
    parser=LanguageParser()
)
documents = loader.load()
len(documents)

In [ ]:
python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, 
                                                               chunk_size=2000, 
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)
len(texts)


EMBEDDING

In [ ]:
embeddings =  OpenAIEmbeddings(disallowed_special=())

In [ ]:

db = Chroma.from_documents(texts,embeddings)
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

QA

In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)